# Worksheet 8

You are encouraged to work in groups of up to 3 total students, but each student should make their own submission on Canvas.  (It's fine for everyone in the group to have the same upload.)

## Overview

This project includes a csv file `unemployment.csv` which includes US unemployment rates for various industries between 2000 and 2010.

* Load that csv file using `pd.read_csv` and save the resulting DataFrame as `df`.

The background question for this homework is,
> How does the time of year affect unemployment in different industries?

In [28]:
import pandas as pd

In [29]:
df = pd.read_csv("unemployment.csv")

## Cleaning the data

* Evaluate `df.dtypes`.  Notice that this "date" column is not being recognized as containing datetime values.

In [3]:
df.dtypes

industry     object
date         object
rate        float64
dtype: object

* Convert this column to datetime format using `pd.to_datetime`.

In [30]:
df["date"]= pd.to_datetime(df["date"])

* Evaluate `df.dtypes` and `df.dtypes["date"]` to make sure the change has actually occurred within `df`.

In [5]:
df.dtypes

industry                 object
date        datetime64[ns, UTC]
rate                    float64
dtype: object

In [6]:
df.dtypes["date"]

datetime64[ns, UTC]

Less good strategy, just for practice with `map` and lambda functions.

* Using the pandas Series method `map` and a lambda function, make a new column `"month0"` which contains the numerical month for each date.  (For example, `4` if the date is in April.)

Side question: why don't you need to use the `dt` accessor here?

In [31]:
df["month0"] = df["date"].map(lambda x : pd.to_datetime(x).month)

In [39]:
df


,industry,date,rate,month0,month
0,Government,2000-01-01 08:00:00+00:00,2.1,1,1
1,Government,2000-02-01 08:00:00+00:00,2.0,2,2
2,Government,2000-03-01 08:00:00+00:00,1.5,3,3
3,Government,2000-04-01 08:00:00+00:00,1.3,4,4
4,Government,2000-05-01 07:00:00+00:00,1.9,5,5
...,...,...,...,...,...
1703,Self-employed,2009-10-01 07:00:00+00:00,5.9,10,10
1704,Self-employed,2009-11-01 07:00:00+00:00,5.7,11,11
1705,Self-employed,2009-12-01 08:00:00+00:00,5.9,12,12
1706,Self-employed,2010-01-01 08:00:00+00:00,7.2,1,1


Better strategy.

* Make a new column `"month"` in the DataFrame which contains the numerical month for each date.  Use the `dt` accessor but not `map`.

In [32]:
df["month"] = df["date"].dt.month

In [10]:
df.head()

,industry,date,rate,month0,month
0,Government,2000-01-01 08:00:00+00:00,2.1,1,1
1,Government,2000-02-01 08:00:00+00:00,2.0,2,2
2,Government,2000-03-01 08:00:00+00:00,1.5,3,3
3,Government,2000-04-01 08:00:00+00:00,1.3,4,4
4,Government,2000-05-01 07:00:00+00:00,1.9,5,5


In [11]:
df

,industry,date,rate,month0,month
0,Government,2000-01-01 08:00:00+00:00,2.1,1,1
1,Government,2000-02-01 08:00:00+00:00,2.0,2,2
2,Government,2000-03-01 08:00:00+00:00,1.5,3,3
3,Government,2000-04-01 08:00:00+00:00,1.3,4,4
4,Government,2000-05-01 07:00:00+00:00,1.9,5,5
...,...,...,...,...,...
1703,Self-employed,2009-10-01 07:00:00+00:00,5.9,10,10
1704,Self-employed,2009-11-01 07:00:00+00:00,5.7,11,11
1705,Self-employed,2009-12-01 08:00:00+00:00,5.9,12,12
1706,Self-employed,2010-01-01 08:00:00+00:00,7.2,1,1


* Verify that the "month" and "month0" columns contain the same values.  First create a Boolean Series, then call the `all` method.

In [33]:
temp1 = df["month"] == df["month0"]

In [34]:
temp1.all()

True

* Evaluate the following.  What is it telling us about the presence of missing values in `df`?
```
df.isna().any(axis=1).any()
```

In [35]:
df.isna().any(axis=1).any()


False

there are no missing values labeled nan in df.

## Normalizing the data

* Make a pandas Series `mean_ser` containing the average unemployment rate for each industry, using the following code.
```
mean_ser = df.groupby("industry")["rate"].mean()
```

In [41]:
mean_ser = df.groupby("industry")["rate"].mean()

* Make the analogous pandas Series for standard deviation, and name it `std_ser`.

In [42]:
std_ser = df.groupby("industry")["rate"].mean()

* Write a function `make_norm` which takes as input a row of `df` (not a row label but the whole row as a pandas Series) and as output returns the normalized unemployment rate, where by "normalized", mean that you should subtract the mean for that industry and divide by the standard deviation for that industry.  

For example, if `rate` is `7`, the `mean` for the industry is `1.2` and the standard deviation for the industry is `4.3`, then the function should return `(7 - 1.2)/4.3`.

In [17]:
df.head()

,industry,date,rate,month0,month
0,Government,2000-01-01 08:00:00+00:00,2.1,1,1
1,Government,2000-02-01 08:00:00+00:00,2.0,2,2
2,Government,2000-03-01 08:00:00+00:00,1.5,3,3
3,Government,2000-04-01 08:00:00+00:00,1.3,4,4
4,Government,2000-05-01 07:00:00+00:00,1.9,5,5


In [43]:
def make_norm(row) :
    mean = mean_ser[row['industry']]
    std = std_ser[row['industry']]
    return (row["rate"]-mean)/std 

* Using `apply`, the above function `make_norm`, and a suitable `axis` argument, for each row in `df`, normalize the unemployment rate (so that the mean becomes 1 and the standard deviation becomes 1).  We need to use `apply` and not `map` here, because we need to know the industry.  Put the result in a new column in `df` called `"norm_rate"`.

In [44]:
df["norm_rate"] = df.apply(make_norm, axis = 1)

* Using `groupby`, check that the means for the various industries of this new `"norm_rate"` column are all very close to 0, and the standard deviations are all close to 1.

In [20]:
df.groupby("industry")["norm_rate"].mean()

industry
Agriculture                     2.311448e-16
Business services               4.040484e-16
Construction                   -4.258888e-16
Education and Health           -3.039463e-16
Finance                         1.492431e-16
Government                     -2.684556e-16
Information                    -5.642117e-17
Leisure and hospitality         2.984862e-16
Manufacturing                  -2.529852e-16
Mining and Extraction           3.640075e-17
Other                           2.275047e-17
Self-employed                   7.116348e-16
Transportation and Utilities    1.565232e-16
Wholesale and Retail Trade     -2.702756e-16
Name: norm_rate, dtype: float64

In [21]:
df.groupby("industry")["norm_rate"].std()

industry
Agriculture                     0.402609
Business services               0.269165
Construction                    0.486687
Education and Health            0.256553
Finance                         0.348968
Government                      0.266064
Information                     0.367528
Leisure and hospitality         0.193080
Manufacturing                   0.415228
Mining and Extraction           0.578248
Other                           0.258491
Self-employed                   0.370858
Transportation and Utilities    0.355269
Wholesale and Retail Trade      0.253784
Name: norm_rate, dtype: float64

## Plotting the data

* Make an Altair chart of this data, using `mark_line`, using "date" for the x-channel, using "norm_rate" for the y-channel, and using "industry" for the color.

Side question: can you recognize the impact of the 2008 financial crisis?

In [22]:
import altair as alt

In [47]:
alt.Chart(df).mark_line().encode(
    x = "date",
    y = "norm_rate",
    color = "industry",
    row = "industry",
    tooltip = "rate"
)

alt.Chart(...)

* Make the same Altair chart, but change from "date" to "month" for the x-channel.

(It will look like a mess because each industry has the same month repeated many times, corresponding to different years.)

In [24]:
alt.Chart(df).mark_line().encode(
    x = "month",
    y = "norm_rate",
    color = "industry"
)

alt.Chart(...)

* Make the same chart as in the previous cell, but change from `y="norm_rate"` to `y="mean(norm_rate)"`.  This will replace the many-points-per-industry with a single point per industry.  Store the chart with the variable name `c`, and display this chart.

In [25]:
c = alt.Chart(df).mark_line().encode(
    x = "month",
    y = "mean(norm_rate)",
    color = "industry"
)

In [26]:
c

alt.Chart(...)

* Notice that "Government" appears to have the highest average normalized unemployment in July.  Using Boolean indexing and `mean`, compute this average directly using pandas, and make sure it matches the value you see in Altair.  (It should be approximately 1.3.)

In [27]:
with open("chart.json", "w") as f:
    f.write(c.to_json())

## Submission

* Save the chart as a json file using the following code and upload that json file to Canvas.
```
with open("chart.json", "w") as f:
    f.write(c.to_json())
```

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7d8b584c-f20e-4772-bb70-373531e57ec2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>